**Importing required libraries**

In [80]:
import os
import numpy as np

from tensorflow import keras  
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten

from pathlib import Path

**Loading the pre-trained VGG16 model** <br>

**Note:** The VGG16 model will be loaded without the top layer, ie the classification layer.

In [81]:
model = VGG16(weights='imagenet', include_top=False)

In [82]:
model.summary()

**Freezing layers** <br>
**Observation:** Keras contains a parameter that indicates whether a particular layer is trainable or not. In order for the weights to be frozen while training the new model, we need to set the workable status parameter to False.

In [83]:
for layer in model.layers[:]:
    layer.trainable = False

for layer in model.layers:
    print(layer, layer.trainable)

**Load *MSI vs MSS* dataset**

In [84]:
PATH = '../input/train-val-test-tcga-coad-msi-mss/tcga_coad_msi_mss/'
train_dir = os.path.join(PATH, 'train')
val_dir = os.path.join(PATH, 'val')

**Data preprocessing**

In [85]:
train_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

In [86]:
train_generator = train_data_generator.flow_from_directory(train_dir,
                                               target_size=(150, 150),
                                               class_mode='binary',
                                               subset='training',
                                               batch_size=20)

In [87]:
valid_generator = test_data_generator.flow_from_directory(val_dir,
                                               target_size=(150, 150),
                                               class_mode='binary',
                                               subset='training',
                                               batch_size=20)

**Construction of the AlexNet model**

In [89]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [92]:
def create_model(input_shape, optimizer='adam', fine_tune=2):
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False


    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(1, activation='sigmoid')(top_model)
    
    model = Model(inputs=conv_base.input, outputs=output_layer)


    model.compile(optimizer='adam', 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [101]:
input_shape = (150, 150, 3)
optim_1 = Adam(learning_rate=0.0001)

n_steps = train_generator.samples // 20
n_val_steps = valid_generator.samples // 20
n_epochs = 30

model = create_model(input_shape, optim_1, fine_tune=2)

In [102]:
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

early_stop = EarlyStopping(monitor='val_loss',
                           patience=1,
                           restore_best_weights=True,
                           mode='min')

In [103]:
model.fit(train_generator,
              batch_size=20,
              epochs=n_epochs,
              validation_data=valid_generator,
              steps_per_epoch=n_steps,
              validation_steps=n_val_steps)

In [109]:
from sklearn import metrics 

In [110]:
steps = np.math.ceil(valid_generator.samples /valid_generator.batch_size)

predicoes = model.predict(valid_generator, steps=steps)
predicao_classes = np.argmax(predicoes, axis=1)
X_classes = valid_generator.classes
classes_rotuladas = list(valid_generator.class_indices.keys())
report = metrics.classification_report(X_classes, predicao_classes, target_names=classes_rotuladas)
print(report)

In [137]:
metrics.f1_score(X_classes, predicao_classes,average='weighted',labels=np.unique(predicao_classes))